In [1]:
%load_ext autoreload
%autoreload 2 
import matplotlib.pyplot as plt
import numpy as np
from numpy import mean
from numpy import std
import pandas as pd
from collections import Counter
import ReAnalysisData as rad

from sklearn import preprocessing, svm
from sklearn import metrics

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate

from sklearn.metrics import accuracy_score

from sklearn.feature_selection import SelectFromModel

In [2]:
columns_new=['Site', 
             'EVI (3 Years Before)', 'NDVI (3 Years Before)', 'FPAR (3 Years Before)', 'LAI (3 Years Before)', 
          'LST Day (3 Years Before)', 'LST Night (3 Years Before)', 'EVI (2 Years Before)', 'NDVI (2 Years Before)', 
          'FPAR (2 Years Before)', 'LAI (2 Years Before)', 'LST Day (2 Years Before)','LST Night (2 Years Before)', 
          'EVI (1 Year Before)', 'NDVI (1 Year Before)', 'FPAR (1 Year Before)', 'LAI (1 Year Before)', 
          'LST Day (1 Year Before)', 'LST Night (1 Year Before)',
       'LATITUDE', 'LONGITUDE', 'FIRE_SIZE', 'FIRE_SIZE_CLASS', 'Date', '_classes']

In [3]:
df = pd.read_csv("Data/wildfire_classes2.csv").dropna()
df.columns = columns_new

In [4]:
positions = df.index
positions

RangeIndex(start=0, stop=2060, step=1)

In [5]:
df['date'] = pd.to_datetime(df['Date'], format='%m/%d/%Y') + pd.to_timedelta('00:00:00')
df.set_index("date", inplace = True)
df

,Site,EVI (3 Years Before),NDVI (3 Years Before),FPAR (3 Years Before),LAI (3 Years Before),LST Day (3 Years Before),LST Night (3 Years Before),EVI (2 Years Before),NDVI (2 Years Before),FPAR (2 Years Before),...,FPAR (1 Year Before),LAI (1 Year Before),LST Day (1 Year Before),LST Night (1 Year Before),LATITUDE,LONGITUDE,FIRE_SIZE,FIRE_SIZE_CLASS,Date,_classes
date,,,,,,,,,,,,,,,,,,,,,
2020-12-24,SOL68,0.072462,0.145540,0.020188,-0.093232,6.862729,5.712826,0.093315,0.174263,0.088271,...,0.083790,0.085731,5.693096,5.559765,33.402651,-117.266807,4276.00,4,12/24/2020,1
2020-11-19,SAD11,-0.010497,-0.041648,-0.003806,0.206635,-2.202834,-0.359681,0.001172,-0.031839,0.002453,...,-0.019707,0.033731,-2.452884,-0.718174,37.306118,-82.758060,281.00,3,11/19/2020,0
2020-11-16,SAD10,-0.020340,-0.033043,-0.021513,0.004912,-2.238707,-1.183263,-0.008354,-0.024423,-0.003541,...,-0.025796,-0.054157,-2.202367,-1.510989,37.299220,-83.089330,281.00,3,11/16/2020,0
2020-11-15,SWA6,0.030746,0.061049,0.035031,0.224315,-0.351470,0.797888,0.032567,0.062465,0.056059,...,0.052183,0.378532,-0.398460,1.044637,35.631146,-94.092307,0.25,0,11/15/2020,0
2020-11-14,SWA1,0.010226,0.022993,0.026851,0.220282,0.042761,0.228278,-0.002914,0.024192,0.038041,...,0.046405,0.356407,-0.161821,0.239998,35.947547,-92.160007,0.25,0,11/14/2020,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2003-08-06,NWA40,-0.079899,-0.164034,-0.096221,-0.369251,-18.060465,-10.428125,-0.051314,-0.100407,-0.057509,...,-0.080047,-0.328383,-6.179146,-3.984859,45.089167,-115.545000,0.25,0,8/6/2003,0
2003-07-08,NWA45,-0.041561,-0.087780,0.153177,0.297926,-12.645636,-8.235142,-0.007655,-0.006803,0.190868,...,0.164146,0.387357,-4.424932,-1.684518,45.054722,-115.716389,0.25,0,7/8/2003,0
2003-04-01,SWA24,0.039704,0.122278,0.112268,0.575155,-11.807328,-2.615032,0.001687,0.095537,0.134972,...,0.094943,0.467796,-3.060271,-0.800667,34.795556,-93.856389,0.25,0,4/1/2003,0


In [6]:
import pandas as pd
import numpy as np
import random
import datetime
import os
from datetime import timedelta
import cdsapi
import datetime
import cdsapi

import ecmwflibs
import eccodes
from eccodes import *

from statistics import mean 


RAD_ROOT_DIR = "data/rad/"
cdsApi = cdsapi.Client(verify=True)


data = []

In [7]:
def getPressureData(cdsApi, year, month, date, time, lati, long, fileName):
    spread = 5
    if os.path.isfile(fileName):
        return
    cdsApi.retrieve(
        'reanalysis-era5-pressure-levels',
        {
            'product_type': 'reanalysis',
            'variable': [
                'geopotential', 'relative_humidity', 'temperature',
                'u_component_of_wind', 'v_component_of_wind',
            ],
            'pressure_level': [
                '300', '500', '700', '850',
            ],
            'year': year,
            'month': month,
            'day': date,
            'time': time,
            'format': 'grib',
            'area': [
                lati + spread, long - spread, lati - spread,
                long + spread,
            ],
        },
        fileName
    )

In [8]:
def read_grib_file(fileName, datac):
    id = codes_index_new_from_file(fileName, ["shortName",'level'])
    names = codes_index_get(id, "shortName")
    levels = codes_index_get(id, "level")
    for level in levels:
        #Getting index for each level 225, 500, 700
        codes_index_select(id, "level", level)
        for name in names:
          #Getting index for each name (u, v, z)
          codes_index_select(id, "shortName", name)
          gid = codes_new_from_index(id)
          values = codes_get_values(gid)
          length = len(values)
          #print(f"gid:{gid}, length:{length}")
          datac.append(mean(values))
          length = len(datac)
    return id, datac

In [9]:
for i in positions:
    fileName = RAD_ROOT_DIR + str(i).zfill(6) + '.grib'
    _trial = df.iloc[i:i+1]

    _strTime = _trial.index.strftime("%Y-%m-%d %H:%M:%S")
    year = _strTime[0][:4]
    month = _strTime[0][5:7]
    date = _strTime[0][8:10]
    time = _strTime[0][11:]

    lati = _trial["LATITUDE"].values[0]
    long = _trial["LONGITUDE"].values[0]
    
    getPressureData(cdsApi, year, month, date, time, lati, long, fileName)
    id, datac = read_grib_file(fileName, [])
    
    datac.extend([lati, long])
    data.append(datac)

2023-09-05 18:09:22,255 INFO Welcome to the CDS
2023-09-05 18:09:22,255 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-pressure-levels
2023-09-05 18:09:22,446 INFO Request is completed
2023-09-05 18:09:22,446 INFO Downloading https://download-0008-clone.copernicus-climate.eu/cache-compute-0008/cache/data2/adaptor.mars.internal-1693881263.5545568-2276-4-c632d69b-c286-402c-82c6-62ab854199ca.grib to data/rad/000000.grib (68K)
2023-09-05 18:09:23,581 INFO Download rate 59.9K/s  
2023-09-05 18:09:24,065 INFO Welcome to the CDS
2023-09-05 18:09:24,065 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-pressure-levels
2023-09-05 18:09:24,246 INFO Downloading https://download-0000-clone.copernicus-climate.eu/cache-compute-0000/cache/data7/adaptor.mars.internal-1693955258.3543355-24840-4-abc96721-f7ca-4829-bfab-adf51587bc20.grib to data/rad/000001.grib (68K)
2023-09-05 18:09:25,182 INFO Download rate 72.6K/s  
2

2023-09-05 18:10:06,733 INFO Download rate 59.9K/s  
2023-09-05 18:10:07,147 INFO Welcome to the CDS
2023-09-05 18:10:07,149 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-pressure-levels
2023-09-05 18:10:07,341 INFO Request is queued
2023-09-05 18:10:08,495 INFO Request is running
2023-09-05 18:10:12,536 INFO Request is completed
2023-09-05 18:10:12,536 INFO Downloading https://download-0004-clone.copernicus-climate.eu/cache-compute-0004/cache/data7/adaptor.mars.internal-1693955406.9107966-11116-8-a14b83e3-cd46-458d-a84e-33367f083aab.grib to data/rad/000016.grib (68K)
2023-09-05 18:10:12,984 INFO Download rate 152.1K/s 
2023-09-05 18:10:13,379 INFO Welcome to the CDS
2023-09-05 18:10:13,380 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-pressure-levels
2023-09-05 18:10:13,582 INFO Request is queued
2023-09-05 18:10:14,729 INFO Request is running
2023-09-05 18:10:22,282 INFO Request is completed
202

2023-09-05 18:11:36,543 INFO Request is queued
2023-09-05 18:11:37,687 INFO Request is running
2023-09-05 18:11:50,460 INFO Request is completed
2023-09-05 18:11:50,460 INFO Downloading https://download-0007-clone.copernicus-climate.eu/cache-compute-0007/cache/data9/adaptor.mars.internal-1693955496.0825593-30513-7-266bf6bc-d579-400b-8df0-5999b9a573b9.grib to data/rad/000029.grib (68K)
2023-09-05 18:11:51,431 INFO Download rate 70K/s    
2023-09-05 18:11:51,859 INFO Welcome to the CDS
2023-09-05 18:11:51,861 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-pressure-levels
2023-09-05 18:11:52,022 INFO Request is queued
2023-09-05 18:11:53,168 INFO Request is running
2023-09-05 18:11:57,204 INFO Request is completed
2023-09-05 18:11:57,204 INFO Downloading https://download-0004-clone.copernicus-climate.eu/cache-compute-0004/cache/data7/adaptor.mars.internal-1693955511.4691477-12028-11-a8d8fddb-26f2-4637-9b92-757fa71aae4c.grib to data/rad/000030.gr

2023-09-05 18:13:15,632 INFO Download rate 164.6K/s 
2023-09-05 18:13:16,012 INFO Welcome to the CDS
2023-09-05 18:13:16,012 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-pressure-levels
2023-09-05 18:13:16,185 INFO Request is queued
2023-09-05 18:13:17,322 INFO Request is running
2023-09-05 18:13:21,377 INFO Request is completed
2023-09-05 18:13:21,377 INFO Downloading https://download-0004-clone.copernicus-climate.eu/cache-compute-0004/cache/data4/adaptor.mars.internal-1693955595.7749517-13251-5-f67747dc-ded5-46b4-a16f-5b2c9489f241.grib to data/rad/000043.grib (68K)
2023-09-05 18:13:22,232 INFO Download rate 79.5K/s  
2023-09-05 18:13:22,576 INFO Welcome to the CDS
2023-09-05 18:13:22,576 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-pressure-levels
2023-09-05 18:13:22,797 INFO Request is queued
2023-09-05 18:13:23,929 INFO Request is running
2023-09-05 18:13:27,960 INFO Request is completed
202

2023-09-05 18:14:43,260 INFO Request is queued
2023-09-05 18:14:44,409 INFO Request is running
2023-09-05 18:14:48,455 INFO Request is completed
2023-09-05 18:14:48,455 INFO Downloading https://download-0003-clone.copernicus-climate.eu/cache-compute-0003/cache/data7/adaptor.mars.internal-1693955682.8740518-31212-18-09d9ade8-c753-42be-ad2f-be498bce3702.grib to data/rad/000056.grib (68K)
2023-09-05 18:14:49,586 INFO Download rate 60.1K/s  
2023-09-05 18:14:49,964 INFO Welcome to the CDS
2023-09-05 18:14:49,965 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-pressure-levels
2023-09-05 18:14:50,142 INFO Request is queued
2023-09-05 18:14:51,283 INFO Request is running
2023-09-05 18:14:55,334 INFO Request is completed
2023-09-05 18:14:55,334 INFO Downloading https://download-0011-clone.copernicus-climate.eu/cache-compute-0011/cache/data7/adaptor.mars.internal-1693955689.7722545-24401-2-2d46e296-0263-45da-87df-e1ef78b952bf.grib to data/rad/000057.gr

...

2023-09-05 22:06:47,499 INFO Download rate 117.4K/s 
2023-09-05 22:06:47,837 INFO Welcome to the CDS
2023-09-05 22:06:47,837 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-pressure-levels
2023-09-05 22:06:48,021 INFO Request is queued
2023-09-05 22:06:49,177 INFO Request is running
2023-09-05 22:06:53,231 INFO Request is completed
2023-09-05 22:06:53,231 INFO Downloading https://download-0015-clone.copernicus-climate.eu/cache-compute-0015/cache/data1/adaptor.mars.internal-1693969607.6566143-15789-6-bad8d0c6-c822-4323-85d3-b71735343d58.grib to data/rad/001997.grib (68K)
2023-09-05 22:06:54,674 INFO Download rate 47.1K/s  
2023-09-05 22:06:55,037 INFO Welcome to the CDS
2023-09-05 22:06:55,038 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-pressure-levels
2023-09-05 22:06:55,203 INFO Request is queued
2023-09-05 22:06:56,360 INFO Request is running
2023-09-05 22:07:00,463 INFO Request is completed
202

2023-09-05 22:08:20,690 INFO Request is queued
2023-09-05 22:08:21,834 INFO Request is running
2023-09-05 22:08:25,884 INFO Request is completed
2023-09-05 22:08:25,884 INFO Downloading https://download-0006-clone.copernicus-climate.eu/cache-compute-0006/cache/data5/adaptor.mars.internal-1693969700.1864057-1774-7-ca3ee265-d65d-4ee5-abcb-efd85d5690e6.grib to data/rad/002010.grib (68K)
2023-09-05 22:08:26,650 INFO Download rate 88.7K/s  
2023-09-05 22:08:26,975 INFO Welcome to the CDS
2023-09-05 22:08:26,977 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-pressure-levels
2023-09-05 22:08:27,146 INFO Request is queued
2023-09-05 22:08:28,280 INFO Request is running
2023-09-05 22:08:32,335 INFO Request is completed
2023-09-05 22:08:32,336 INFO Downloading https://download-0007-clone.copernicus-climate.eu/cache-compute-0007/cache/data1/adaptor.mars.internal-1693969706.9663475-2912-17-6caa36a6-0727-41c8-beca-98b18a854aa6.grib to data/rad/002011.grib

2023-09-05 22:09:55,807 INFO Download rate 61.7K/s  
2023-09-05 22:09:56,128 INFO Welcome to the CDS
2023-09-05 22:09:56,128 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-pressure-levels
2023-09-05 22:09:56,336 INFO Request is queued
2023-09-05 22:09:57,508 INFO Request is running
2023-09-05 22:10:01,558 INFO Request is completed
2023-09-05 22:10:01,558 INFO Downloading https://download-0021.copernicus-climate.eu/cache-compute-0021/cache/data6/adaptor.mars.internal-1693969796.0567372-11615-16-8bd733da-76d7-4bdd-8b6d-4b2f63c4735f.grib to data/rad/002024.grib (68K)
2023-09-05 22:10:02,650 INFO Download rate 62.2K/s  
2023-09-05 22:10:02,979 INFO Welcome to the CDS
2023-09-05 22:10:02,979 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-pressure-levels
2023-09-05 22:10:03,165 INFO Request is queued
2023-09-05 22:10:04,318 INFO Request is running
2023-09-05 22:10:08,354 INFO Request is completed
2023-09-

2023-09-05 22:11:28,261 INFO Request is queued
2023-09-05 22:11:29,430 INFO Request is running
2023-09-05 22:11:33,477 INFO Request is completed
2023-09-05 22:11:33,477 INFO Downloading https://download-0017.copernicus-climate.eu/cache-compute-0017/cache/data3/adaptor.mars.internal-1693969887.960588-3116-19-db1875d8-c8b7-4f5d-852b-bf03cfe6a1fa.grib to data/rad/002037.grib (68K)
2023-09-05 22:11:33,926 INFO Download rate 151.2K/s 
2023-09-05 22:11:34,261 INFO Welcome to the CDS
2023-09-05 22:11:34,261 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-pressure-levels
2023-09-05 22:11:34,435 INFO Request is queued
2023-09-05 22:11:35,570 INFO Request is running
2023-09-05 22:11:39,608 INFO Request is completed
2023-09-05 22:11:39,608 INFO Downloading https://download-0009-clone.copernicus-climate.eu/cache-compute-0009/cache/data9/adaptor.mars.internal-1693969894.1760247-20843-16-e03882b8-d830-4d8a-8e75-caf1e966c0e5.grib to data/rad/002038.grib (68K

2023-09-05 22:13:03,827 INFO Download rate 131K/s   
2023-09-05 22:13:04,228 INFO Welcome to the CDS
2023-09-05 22:13:04,228 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-pressure-levels
2023-09-05 22:13:04,471 INFO Request is queued
2023-09-05 22:13:05,637 INFO Request is running
2023-09-05 22:13:09,685 INFO Request is completed
2023-09-05 22:13:09,685 INFO Downloading https://download-0019.copernicus-climate.eu/cache-compute-0019/cache/data0/adaptor.mars.internal-1693969984.022362-5561-16-853b2ddb-3b75-4e9c-a165-6455b0b7fd2b.grib to data/rad/002051.grib (68K)
2023-09-05 22:13:10,240 INFO Download rate 122.3K/s 
2023-09-05 22:13:10,626 INFO Welcome to the CDS
2023-09-05 22:13:10,642 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-pressure-levels
2023-09-05 22:13:10,835 INFO Downloading https://download-0019.copernicus-climate.eu/cache-compute-0019/cache/data0/adaptor.mars.internal-1693969984.022362

In [11]:
wf_era5 = pd.DataFrame(data)
wf_era5.to_csv("wf_era5.csv")